In [1]:
# Definir o caminho do bucket GCS
bucket_path = "gs://pos-graduacao/Olimpiadas/"

# Carregar os arquivos CSV em DataFrames
df_athletes = spark.read.csv(bucket_path + "olympic_athletes.csv", header=True, inferSchema=True)
df_hosts = spark.read.csv(bucket_path + "olympic_hosts.csv", header=True, inferSchema=True)
df_medals = spark.read.csv(bucket_path + "olympic_medals.csv", header=True, inferSchema=True)
df_results = spark.read.csv(bucket_path + "olympic_results.csv", header=True, inferSchema=True)

In [2]:
df_athletes.createOrReplaceTempView("olympic_athletes")
df_hosts.createOrReplaceTempView("olympic_hosts")
df_medals.createOrReplaceTempView("olympic_medals")
df_results.createOrReplaceTempView("olympic_results")

In [3]:
df_athletes.show(5)
df_hosts.show(5)
df_medals.show(5)
df_results.show(5)

+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|         athlete_url|   athlete_full_name|games_participations|      first_game|athlete_year_birth|athlete_medals| bio|
+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|https://olympics....|Cooper WOODS-TOPA...|                   1|    Beijing 2022|              2000|          NULL|NULL|
|https://olympics....|      Felix ELOFSSON|                   2|PyeongChang 2018|              1995|          NULL|NULL|
|https://olympics....|       Dylan WALCZYK|                   1|    Beijing 2022|              1993|          NULL|NULL|
|https://olympics....|       Olli PENTTALA|                   1|    Beijing 2022|              1995|          NULL|NULL|
|https://olympics....|    Dmitriy REIKHERD|                   1|    Beijing 2022|              1989|          NULL|NULL|
+--------------------+----------

In [4]:
top_3_medals_query = spark.sql("""
    WITH ranked_medals AS (
        SELECT h.game_year, h.game_location, m.country_name, COUNT(*) AS total_medals,
               RANK() OVER (PARTITION BY h.game_year, h.game_location ORDER BY COUNT(*) DESC) AS rnk
        FROM olympic_medals m
        JOIN olympic_hosts h ON m.slug_game = h.game_slug
        GROUP BY h.game_year, h.game_location, m.country_name
    )
    SELECT game_year, game_location, country_name, total_medals
    FROM ranked_medals
    WHERE rnk <= 3
    ORDER BY game_year, game_location, rnk
""")
top_3_medals_query.show()


+---------+-------------+--------------------+------------+
|game_year|game_location|        country_name|total_medals|
+---------+-------------+--------------------+------------+
|     1896|       Greece|              Greece|          49|
|     1896|       Greece|United States of ...|          20|
|     1896|       Greece|             Germany|          13|
|     1900|       France|              France|         103|
|     1900|       France|United States of ...|          48|
|     1900|       France|       Great Britain|          35|
|     1904|United States|United States of ...|         241|
|     1904|United States|             Germany|          15|
|     1904|United States|              Canada|           6|
|     1908|Great Britain|       Great Britain|         160|
|     1908|Great Britain|United States of ...|          47|
|     1908|Great Britain|              Sweden|          26|
|     1912|       Sweden|              Sweden|          68|
|     1912|       Sweden|United States o

In [5]:
avg_medals_query = spark.sql("""
    SELECT m.game_year, m.game_location, m.country_name, AVG(m.medal_count) AS avg_medals
    FROM (
        SELECT h.game_year, h.game_location, m.country_name, COUNT(*) AS medal_count
        FROM olympic_medals m
        JOIN olympic_hosts h ON m.slug_game = h.game_slug
        GROUP BY h.game_year, h.game_location, m.country_name, m.slug_game
    ) m
    GROUP BY m.game_year, m.game_location, m.country_name
    ORDER BY m.game_year, avg_medals DESC
""")
avg_medals_query.show()


+---------+-------------+--------------------+----------+
|game_year|game_location|        country_name|avg_medals|
+---------+-------------+--------------------+----------+
|     1896|       Greece|              Greece|      49.0|
|     1896|       Greece|United States of ...|      20.0|
|     1896|       Greece|             Germany|      13.0|
|     1896|       Greece|              France|      11.0|
|     1896|       Greece|       Great Britain|       7.0|
|     1896|       Greece|             Denmark|       6.0|
|     1896|       Greece|             Hungary|       6.0|
|     1896|       Greece|             Austria|       5.0|
|     1896|       Greece|                 MIX|       4.0|
|     1896|       Greece|         Switzerland|       3.0|
|     1896|       Greece|           Australia|       2.0|
|     1900|       France|              France|     103.0|
|     1900|       France|United States of ...|      48.0|
|     1900|       France|       Great Britain|      35.0|
|     1900|   

In [6]:
host_performance_query = spark.sql("""
    SELECT game_year, game_location, AVG(m.total_medals) AS avg_medals
    FROM (
        SELECT h.game_year, h.game_location, m.country_name, COUNT(*) AS total_medals
        FROM olympic_medals m
        JOIN olympic_hosts h ON m.slug_game = h.game_slug
        GROUP BY h.game_year, h.game_location, m.country_name
    ) m
    GROUP BY game_year, game_location
    ORDER BY game_year
""")
host_performance_query.show()


+---------+-----------------+------------------+
|game_year|    game_location|        avg_medals|
+---------+-----------------+------------------+
|     1896|           Greece|11.454545454545455|
|     1900|           France|13.904761904761905|
|     1904|    United States|22.307692307692307|
|     1908|    Great Britain| 18.05263157894737|
|     1912|           Sweden|17.944444444444443|
|     1920|          Belgium|20.772727272727273|
|     1924|           France|16.555555555555557|
|     1928|      Switzerland|3.6666666666666665|
|     1928|      Netherlands|10.090909090909092|
|     1932|    United States|              14.5|
|     1936|          Germany|          13.90625|
|     1948|      Switzerland|5.6923076923076925|
|     1948|    Great Britain|11.837837837837839|
|     1952|           Norway| 5.615384615384615|
|     1952|          Finland|10.813953488372093|
|     1956|            Italy|               6.0|
|     1956|Australia, Sweden| 12.91891891891892|
|     1960|         

In [7]:
event_medals_query = spark.sql("""
    SELECT game_year, event_title, country_name, AVG(total_medals) AS avg_medals
    FROM (
        SELECT h.game_year, r.event_title, m.country_name, COUNT(*) AS total_medals
        FROM olympic_results r
        JOIN olympic_medals m ON r.athlete_url = m.athlete_url
        JOIN olympic_hosts h ON r.slug_game = h.game_slug
        GROUP BY h.game_year, r.event_title, m.country_name
    ) m
    GROUP BY game_year, event_title, country_name
    ORDER BY game_year, avg_medals DESC
""")
event_medals_query.show()


+---------+--------------------+------------+----------+
|game_year|         event_title|country_name|avg_medals|
+---------+--------------------+------------+----------+
|     1896|       long jump men|     Germany|      10.0|
|     1896|        shot put men|     Germany|      10.0|
|     1896|    pommel horse men|     Germany|       8.0|
|     1896|           vault men|     Germany|       8.0|
|     1896|  horizontal bar men|     Germany|       8.0|
|     1896|           rings men|     Germany|       8.0|
|     1896|   parallel bars men|     Germany|       8.0|
|     1896|        shot put men|     Denmark|       6.0|
|     1896|Sprint individual...|      France|       6.0|
|     1896|     triple jump men|     Germany|       6.0|
|     1896|    discus throw men|     Denmark|       6.0|
|     1896|  1km time trial men|      France|       6.0|
|     1896|            10km men|      France|       6.0|
|     1896| army rifle 200m men|     Denmark|       6.0|
|     1896|            100m men

In [8]:
df_temp = spark.sql("""
    SELECT h.game_year, a.athlete_year_birth,
           ((h.game_year - a.athlete_year_birth) / 5) * 5 AS start_age_group,
           ((h.game_year - a.athlete_year_birth) / 5) * 5 + 4 AS end_age_group
    FROM olympic_athletes a
    JOIN olympic_medals m ON a.athlete_url = m.athlete_url
    JOIN olympic_hosts h ON m.slug_game = h.game_slug
""")

df_temp.show()


+---------+------------------+---------------+-------------+
|game_year|athlete_year_birth|start_age_group|end_age_group|
+---------+------------------+---------------+-------------+
|     2018|              1994|           24.0|         28.0|
|     2022|              1997|           25.0|         29.0|
|     2018|              1997|           21.0|         25.0|
|     2014|              1992|           22.0|         26.0|
|     2018|              1992|           26.0|         30.0|
|     2022|              1992|           30.0|         34.0|
|     2022|              2000|           22.0|         26.0|
|     2014|              1989|           25.0|         29.0|
|     2018|              2001|           17.0|         21.0|
|     2022|              2001|           21.0|         25.0|
|     2014|              1990|           24.0|         28.0|
|     2018|              1990|           28.0|         32.0|
|     2022|              1990|           32.0|         36.0|
|     2018|             

In [9]:
age_group_medals_query = spark.sql("""
    SELECT h.game_year, CONCAT(FLOOR((h.game_year - a.athlete_year_birth) / 5) * 5, '-', 
           FLOOR((h.game_year - a.athlete_year_birth) / 5) * 5 + 4) AS age_group, COUNT(*) AS medals_count 
    FROM olympic_athletes a 
    JOIN olympic_medals m ON a.athlete_url = m.athlete_url 
    JOIN olympic_hosts h ON m.slug_game = h.game_slug 
    GROUP BY h.game_year, age_group 
    ORDER BY h.game_year, age_group
""")
age_group_medals_query.show()


+---------+---------+------------+
|game_year|age_group|medals_count|
+---------+---------+------------+
|     1896|     NULL|          17|
|     1896|    10-14|           1|
|     1896|    15-19|           4|
|     1896|    20-24|          43|
|     1896|    25-29|          14|
|     1896|    30-34|          10|
|     1900|     NULL|          37|
|     1900|    -5--1|           2|
|     1900|    15-19|          10|
|     1900|    20-24|          54|
|     1900|    25-29|          35|
|     1900|    30-34|          24|
|     1900|    35-39|           5|
|     1900|    40-44|           2|
|     1900|    45-49|           1|
|     1904|     NULL|          37|
|     1904|  -80--76|           1|
|     1904|    15-19|          19|
|     1904|    20-24|          64|
|     1904|    25-29|          37|
+---------+---------+------------+
only showing top 20 rows



In [13]:
#País com mais medalhas por tipo de evento
event_top_country_query = spark.sql("""
    SELECT r.event_title, m.country_name, COUNT(*) AS total_medals
    FROM olympic_results r
    JOIN olympic_medals m ON r.athlete_url = m.athlete_url
    GROUP BY r.event_title, m.country_name
    ORDER BY total_medals DESC
""")
event_top_country_query.show()


+--------------------+--------------------+------------+
|         event_title|        country_name|total_medals|
+--------------------+--------------------+------------+
|individual all-ro...|               Japan|         188|
|individual all-ro...|        Soviet Union|         164|
|individual all-ro...|        Soviet Union|         148|
| floor exercises men|               Japan|         146|
|  horizontal bar men|               Japan|         139|
|           rings men|               Japan|         132|
|           vault men|        Soviet Union|         129|
|           5000m men|         Netherlands|         126|
|floor exercises w...|        Soviet Union|         125|
|        shot put men|United States of ...|         124|
|   parallel bars men|               Japan|         124|
|  horizontal bar men|        Soviet Union|         123|
|  200m butterfly men|United States of ...|         122|
|200m individual m...|United States of ...|         119|
|         vault women|        S

In [15]:
#Número de medalhas por gênero dos atletas
gender_medals_query = spark.sql("""
    SELECT m.event_gender, COUNT(*) AS total_medals
    FROM olympic_medals m
    GROUP BY m.event_gender
    ORDER BY total_medals DESC
""")
gender_medals_query.show()


+------------+------------+
|event_gender|total_medals|
+------------+------------+
|         Men|       13932|
|       Women|        6323|
|        Open|         998|
|       Mixed|         444|
+------------+------------+

